In [1]:
import os
import os.path as osp
import json
import numpy as np
import cv2
from PIL import Image
import glob

class MaskToImageConverter:
    """Converts JSON masks in given directories to images with color-coded segmentation."""

    def __init__(self, base_folder, output_base_folder, class_to_color, resolution=(4000, 2250)):
        self.base_folder = base_folder
        self.output_base_folder = output_base_folder
        self.class_to_color = class_to_color
        self.resolution = resolution

        if not osp.exists(self.output_base_folder):
            os.makedirs(self.output_base_folder)

    def convert(self):
        # Loop through each sub-folder (Test, Train, Val)
        for subset in ['Test', 'Train', 'Val']:
            subset_folder = osp.join(self.base_folder, subset)
            output_folder = osp.join(self.output_base_folder, subset)
            
            if not osp.exists(output_folder):
                os.makedirs(output_folder)
            
            # Process each JSON file in the subset folder
            json_list = glob.glob(osp.join(subset_folder, '*.json'))
            
            for json_file in json_list:
                self.process_file(json_file, output_folder)

    def process_file(self, json_file, output_folder):
        file_name = osp.splitext(osp.basename(json_file))[0]

        # Load the JSON file containing the shapes
        with open(json_file, 'r') as file:
            gt_json = json.load(file)

        # Create a blank image for labels
        label_img = np.zeros((self.resolution[1], self.resolution[0], 3), dtype=np.uint8)

        for shape in gt_json['shapes']:
            points_np = np.array(shape['points'], dtype=np.float64)
            label = shape['label']
            if label not in self.class_to_color:
                raise KeyError(f"Label '{label}' not found in class_to_color")

            color = self.class_to_color[label]
            points_np[:, 0] *= self.resolution[0] / gt_json['imageWidth']
            points_np[:, 1] *= self.resolution[1] / gt_json['imageHeight']
            points_np = np.round(points_np).astype(np.int64)
            points_np = points_np.reshape((-1, 1, 2))
            cv2.fillPoly(label_img, [points_np], color)

        label_img_path = osp.join(output_folder, file_name + '.png')
        Image.fromarray(label_img).save(label_img_path)

# Define the class-to-color mapping and directories
class_to_color = {'Doenca': (255, 0, 0), 'Saudavel': (0, 255, 0), 'Solo': (0, 0, 255)}
base_folder = 'E:/Mestrado/Materias/2024-02-Sistemas para Processamento Multimidia/seminario/image-segmentation/dataset/base'
output_base_folder = 'E:/Mestrado/Materias/2024-02-Sistemas para Processamento Multimidia/seminario/image-segmentation/dataset/masks'

# Create an instance of the converter with the new resolution and process all files
converter = MaskToImageConverter(base_folder, output_base_folder, class_to_color)
converter.convert()